In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, make_scorer

# Configuration de MLflow
# On donne un nom à l'expérience pour s'y retrouver
mlflow.set_experiment("Credit_Scoring_Exp")

print("MLflow initialisé !")

c:\Users\aubin\Majeur IA\data analysis\credit-scoring\venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/12/04 16:04:53 INFO mlflow.tracking.fluent: Experiment with name 'Credit_Scoring_Exp' does not exist. Creating a new experiment.


MLflow initialisé !


In [2]:
# On charge les pickles qu'on a sauvegardés à la fin du notebook 01
X_train = pd.read_pickle('../data/X_train.pkl')
X_test = pd.read_pickle('../data/X_test.pkl')
y_train = pd.read_pickle('../data/y_train.pkl')
y_test = pd.read_pickle('../data/y_test.pkl')

print(f"Train : {X_train.shape}, Test : {X_test.shape}")

Train : (246008, 281), Test : (61503, 281)


In [ ]:
def business_cost_metric(y_true, y_pred_proba, threshold=0.5):

    # Convertir probas en classes 0/1 selon le seuil
    y_pred = (y_pred_proba > threshold).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Formule du coût
    cost = 10 * fn + 1 * fp
    
    # On normalise pour avoir un score comparable (optionnel, mais pratique)
    # Plus c'est bas, mieux c'est.
    return cost

In [ ]:
mlflow.set_experiment(experiment_id="0")

mlflow.autolog()
# 1. Création du modèle "Idiot" (Stratégie: prédit la classe majoritaire)
dummy = DummyClassifier(strategy='most_frequent')

# 2. Entraînement
dummy.fit(X_train, y_train)

# 3. Prédictions (Probas)
# Le dummy va mettre 0 partout ou des probas basiques
y_prob = dummy.predict_proba(X_test)[:, 1]

# 4. Calcul des Métriques
auc_score = roc_auc_score(y_test, y_prob)
cost_score = business_cost_metric(y_test, y_prob)

print(f"Dummy AUC: {auc_score:.4f}")     # Devrait être 0.5
print(f"Dummy Cost: {cost_score}")       # Sera élevé car il rate tous les défauts (FN)

# 5. LOGGING MLFLOW (C'est ça qu'on veut !)
# On enregistre les paramètres (quel algo ?)
mlflow.log_param("model_type", "DummyClassifier")
mlflow.log_param("strategy", "most_frequent")

# On enregistre les performances
mlflow.log_metric("auc", auc_score)
mlflow.log_metric("business_cost", cost_score)

# On enregistre le modèle lui-même (le fichier)
mlflow.sklearn.log_model(dummy, "model")

print("Run logged to MLflow!")

2025/12/04 16:14:01 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/12/04 16:14:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ea158a1b1f924f20905b5fe2aad8e216', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/12/04 16:14:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aubin\Majeur IA\data analysis\credit-scoring\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare inte

Dummy AUC: 0.5000
Dummy Cost: 49650


2025/12/04 16:14:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run logged to MLflow!


In [12]:
# modele LightGBM avec gestion du déséquilibre sur le dataset v1
import lightgbm as lgb

start_time = pd.Timestamp.now()
# class_weight='balanced' gère automatiquement le déséquilibre 92/8
lgbm = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Entraînement LightGBM en cours...")
lgbm.fit(X_train, y_train)
end_time = pd.Timestamp.now()
duration = (end_time - start_time).total_seconds()
y_prob = lgbm.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_prob)
cost_score = business_cost_metric(y_test, y_prob)

print(f"LGBM AUC: {auc_score:.2f}")   
print(f"LGBM Cost: {cost_score}")    
print(f"Training Duration: {duration:.2f} seconds")

mlflow.end_run()

Entraînement LightGBM en cours...


2025/12/04 20:12:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aubin\Majeur IA\data analysis\credit-scoring\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/12/04 20:12:37 WARNING mlflow.lightgbm: Failed to log dataset information to MLflow Tracking. Reason: The following fai

LightGBMError: Do not support special JSON characters in feature name.